In [ ]:
import requests
import json

#log in 
username = ""
password = ""

# Define query
query_data = {
    "query": {
        "bool": {
            "must": [
                {
                    "term": {
                        "dokumenter.dokumentMimeType": "application"
                    }
                },
                {
                    "term": {
                        "dokumenter.dokumentMimeType": "xml"
                    }
                },
                {
                    "range": {
                        "offentliggoerelsesTidspunkt": {
                            "gt": "2017-02-21T00:00:00.001Z",
                            "lt": "2022-02-21T23:59:59.999Z"
                        }
                    }
                }
            ],
            "must_not": [],
            "should": []
        }
    },
   "size": 100  # sample
}

# URL to API
url = "http://distribution.virk.dk/offentliggoerelser/_search"

# Get-Request
response = requests.get(url, headers={"Content-Type": "application/json"}, json=query_data, auth=(username, password))

# Status
if response.status_code == 200:
    response_data = response.json()
    print("URL: OK")
elif response.status_code == 500:
    print("EROR")
else:
    print("URL: ERROR", response.status_code)

# URL function
def extract_xbrl_urls(response_data):
    xbrl_urls = []
    for hit in response_data["hits"]["hits"]:
        for document in hit["_source"]["dokumenter"]:
            if document["dokumentMimeType"] == "application/xml":
                xbrl_urls.append(document["dokumentUrl"])
    return xbrl_urls

# Fetch
xbrl_urls = extract_xbrl_urls(response_data)

# Summarize
success_count = 0
error_count = 0

# Download
for xbrl_url in xbrl_urls:
    xbrl_response = requests.get(xbrl_url)
    if xbrl_response.status_code == 200:

        success_count += 1
    else:
        error_count += 1

#Result
print("Antal hentet:", success_count)
print("Antal fejl:", error_count)
    